In [2]:

import numpy as np
import os
import tqdm as tqdm
import pickle

import numpy as np
import os
import spacy
# import ujson as json
import urllib.request

# from args import get_setup_args
from codecs import open
from collections import Counter
from subprocess import run
from tqdm import tqdm
from zipfile import ZipFile



class Embedding_Matrix():
    def __init__(self,embedding_dir = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD"):
#         embedding_dir = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD"
        with open(r"E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD\\dictionaries.pkl", "rb") as input_file:
            dictionaries = pickle.load(input_file)
        self.word_to_index = dictionaries["word_to_index"]
        self.char_to_index = dictionaries["char_to_index"]
        self.index_to_word = dictionaries["index_to_word"]
        self.index_to_char = dictionaries["index_to_char"]
        
        
    def index_files_using_char_to_index(self, filename, _dict, max_words, max_chars):

        f = codecs.open(filename, "r", encoding="utf-8")
        lines = f.readlines()
        lines = [l.lower() for l in lines]
        encoded_lines = []
        for l in lines:
            tokens = l.split()
            tokens = tokens[:max_words]
            encoded_tokens = []
            for t in tokens:
                l = list(t)
                l = l[:max_chars] ## there is a max limit for the length of characters = max_chars
                encoded_chars = []
                for j in l:
                    if j in _dict:
                        encoded_chars.append(_dict[j])
                    else:
                        encoded_chars.append(0)  ## if the character id not in dictionary put '0' in its place
                encoded_tokens.append(encoded_chars)
            encoded_lines.append(encoded_tokens)

        return encoded_lines

    def index_files_using_word_to_index(self, filename, _dict, max_words):
        f = codecs.open(filename, "r", encoding="utf-8")

        lines = f.readlines()
        lines  = [l.lower() for l in lines]
        encoded_lines = []
        for l in lines:
            tokens = l.split()
            tokens = tokens[:max_words]
            temp = []
            for t in tokens:
                if t in _dict:
                    temp.append(_dict[t])
                else:
                    temp.append(1)

            encoded_lines.append(temp[:])

        return encoded_lines
    def index_files_to_char_level_and_word_level(self, datapath = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD", max_words=0, max_chars=0):
#         files = [".context", ".question", ".answer_text"]
        files = [".question", ".answer_text"]


        for f in files:
            read_path_train = os.path.join(datapath, "train" + f)
            write_path_train_word = os.path.join(datapath, "train_word_index" + f + "_pkl.pkl")
            write_path_train_char = os.path.join(datapath, "train_char_index" + f + "_pkl.pkl")

            read_path_valid = os.path.join(datapath, "validation" + f)
            write_path_valid_word = os.path.join(datapath, "validation_word_index" + f + "_pkl.pkl")
            write_path_valid_char = os.path.join(datapath, "validation_char_index" + f + "_pkl.pkl")



            temp_train_word = index_files_using_word_to_index(self, read_path_train, self.word_to_index, max_words)
            temp_valid_word = index_files_using_word_to_index(self, read_path_valid, self.word_to_index, max_words)

            temp_train_char = index_files_using_char_to_index(self, read_path_train, self.char_to_index, max_words,max_chars)
            temp_valid_char = index_files_using_char_to_index(self, read_path_valid, self.char_to_index, max_words,max_chars)

            write_file_train_word = open(write_path_train_word, "wb")
            pickle.dump(temp_train_word, write_file_train_word)

            write_file_train_char = open(write_path_train_char, "wb")
            pickle.dump(temp_train_char, write_file_train_char)

            write_file_valid_word = open(write_path_valid_word, "wb")
            pickle.dump(temp_valid_word, write_file_valid_word)

            write_file_valid_char = open(write_path_valid_char, "wb")
            pickle.dump(temp_valid_char, write_file_valid_char)

    def get_glove_embeddings(self, word_embedding_size = 100, char_embedding_size = 20 , embedding_dir = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD" ):



        glove_embeddings = os.path.join(embedding_dir, "glove_embeddings100.txt")

        glove_embeddings = open(glove_embeddings,'r', encoding = 'utf-8')



        #     glove_embeddings = pickle.load(open(glove_embeddings))

        #####################  CHECK HOW GLOVE EMBEDDINGS WORK ##############
        temp_embeddings = []

        for word in self.word_to_index:

                if word in ['<pad>', '<s>', '<eos>','<unk>']:
                    temp_vector = np.zeros((word_embedding_size))
                elif word not in glove_embeddings:
                    temp_vector = np.random.uniform(-np.sqrt(3)/np.sqrt(word_embedding_size), np.sqrt(3)/np.sqrt(word_embedding_size), word_embedding_size)
                else:
                    temp_vector = glove_embeddings[word]

                temp_embeddings.append(temp_vector)

        temp_embeddings = np.asarray(temp_embeddings)
        temp_embeddings = temp_embeddings.astype(np.float32)
        self.word_embeddings = temp_embeddings


        char_embeddings = []
        print (char_embedding_size)
        char_embeddings.append(np.zeros((char_embedding_size)))

        for i in range(len(self.char_to_index)):
            temp_vector = np.random.uniform(-np.sqrt(3)/np.sqrt(char_embedding_size), np.sqrt(3)/np.sqrt(char_embedding_size), char_embedding_size)
            char_embeddings.append(temp_vector)

        char_embeddings = np.asarray(char_embeddings)
        char_embeddings = char_embeddings.astype(np.float32)

        self.char_embeddings = char_embeddings

        pickle.dump(char_embeddings, open(os.path.join(embedding_dir, "char_embeddings" + ".pkl"), "wb")) 
        pickle.dump(temp_embeddings, open(os.path.join(embedding_dir, "glove_word_embeddings" + ".pkl"), "wb"))


#         return self.word_embeddings, self.char_embeddings




KeyboardInterrupt: 

In [ ]:
embedding = Embedding_Matrix(embedding_dir = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD")
embedding.get_glove_embeddings(word_embedding_size = 100, char_embedding_size = 20 , embedding_dir = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD" )


In [110]:
embedding.index_files_to_char_level_and_word_level(datapath = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD", max_words=400, max_chars=10)

In [3]:
embedding_dir = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD"
with open(r"E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD\\glove_word_embeddings.pkl", "rb") as input_file:
    glove = pickle.load(input_file)

In [13]:
glove[100]

array([-0.06258035,  0.0150595 ,  0.02575215,  0.17270675,  0.07960177,
       -0.00596011,  0.13488224, -0.03397342, -0.1348676 ,  0.07859553,
        0.09202604, -0.03717313, -0.09934395,  0.08046655,  0.16119549,
       -0.02862258, -0.11719763,  0.160727  ,  0.17106102, -0.07843148,
       -0.15625611, -0.10382988,  0.13577586,  0.05128137,  0.11930855,
        0.01463436,  0.1415507 ,  0.04997107, -0.10924181, -0.05069717,
        0.15108474,  0.11705878, -0.05256043,  0.01571849,  0.12796202,
       -0.09618449,  0.1120216 , -0.00436207,  0.06399671, -0.00043768,
       -0.0093321 ,  0.0866193 , -0.14117904,  0.09678099,  0.14233519,
       -0.15919808, -0.07660368, -0.06428248,  0.10527738, -0.04710912,
       -0.07718737,  0.06490774,  0.02479386,  0.14717807,  0.12930167,
        0.01818467,  0.1511378 , -0.06462082, -0.06716185, -0.08174136,
        0.02943872, -0.0541234 ,  0.08440569, -0.11220339, -0.09587555,
       -0.09138144,  0.17025383, -0.167808  ,  0.07598063, -0.05